# Numpy

While Python has a very nice syntax, it is also comparatively slow for certain tasks.  
This doesn't really matter for many applications, but for heavy numeric work it is very annoying.  
Thus the `NumPy` library was created, which interfaces the two fast `C` and `Fortran` languages.  
The core object of `NumPy` is the N-dimensional array called `NDArray`.  
It has a very similar interface to the Python list except being limited to a single data type per array.  

In [ ]:
from __future__ import annotations

from typing import Any, cast

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy.typing import NDArray


def test(x: Any, expected: Any) -> None:
    if isinstance(x, np.ndarray) and isinstance(expected, np.ndarray):
        if not np.array_equal(x, expected):
            raise AssertionError(f"Expected {expected}, got {x}")
    else:
        if x != expected:
            raise AssertionError(f"Expected {expected}, got {x}")
    print("Test passed")


## Creating arrays from Python containers

You can build arrays from Python containers or from `numpy` functions.  
In general the latter is preferred when possible as it is faster, but we will start with the first.  

**Pay special attention to the difference between a row vector and a one-dimensional array** 

In [ ]:
# A one-dimensional array
arr = np.array([1, 2, 3])

print(arr.ndim)  # number of dimensions
print(arr.shape)  # number of elements per dimension
print(arr)


In [ ]:
# a row vector
arr = np.array([[1, 2, 3]])

print(arr.ndim)  # number of dimensions
print(arr.shape)  # number of elements per dimension
print(arr)

In [ ]:
# a column vector
arr = np.array([[1], [2], [3]])

print(arr.ndim)  # number of dimensions
print(arr.shape)  # number of elements per dimension
print(arr)

In [ ]:
# A two-dimensional array (= matrix)
arr = np.array([[1, 2, 3], [4, 5, 6]])

print(arr.ndim)  # number of dimensions
print(arr.shape)  # number of elements per dimension
print(arr)


### Exercise: type coercion / upcasting

You can check the data type of an array with the `.dtype` property.

- What is the data type of `np.array([1, 2, 3])`? Did you expect that?
- What is the data type of `np.array([1, 2, "3"])`? Did you expect that?

You can enforce the data type using the `dtype` argument of `np.array`

- What is the data type of `np.array([1, 2, "3"], dtype=float)`? What happened to the `"3"`?
- Does that always work? What would happen if you change `"3"` to `"a"`?




In [ ]:
print(np.array([1, 2, 3]).dtype)
print(np.array([1, 2, "3"]).dtype)  # all values upcasted to char
print(np.array([1, 2, "3"], dtype=float))  # "3" converted to float

try:
    print(np.array([1, 2, "a"], dtype=float))  # "3" converted to float
except ValueError:
    print("As expected, 'a' cannot be converted to float")


## Creating arrays from numpy functions

Now let's check out some creation routines.  
First we have `arange` and `linspace`.  
`arange(start, stop, step_size)` behaves like the Python `range`, except that it returns an array instead of an iterator.  
Since `arange` can produce floats, you can also set the step to non-integer values.
The third argument for `linspace(start, stop, num)` isn't the step-size as with `range` or `arange`, but the number of values you want to have.  
Also note that `linspace` **includes** the second argument, so it is a inclusive range `[start, stop]`.  
As the name suggests the values between start and stop are spaced **linearly**.  


```python
np.arange(0, 5, 1)       # array([0, 1, 2, 3, 4])
np.linspace(0, 5, 6)     # array([0, 1, 2, 3, 4])
```

There are also `geomspace` and `logspace` for **geometric** spacing between the values.  
The difference between them is that for `logspace` you give `10 ** start` and `10 ** end`.

```python
np.geomspace(1, 100, 3)  # array([  1.,  10., 100.]
np.logspace(0, 2, 3)     # array([  1.,  10., 100.])
```

Other useful array generation functions are `zeros`, `ones` and `full`, which will create arrays of shape `(N, M)` full of `0`, `1` or a value of choice respectively.

```python
np.zeros((3, 3))               # 3x3 
np.ones((3, 3))                # 3x3
np.full((3, 3), fill_value=2)  # 3x3

#  array([[2, 2, 2],
#         [2, 2, 2],
#         [2, 2, 2]])

```

You can also generate arrays filled with *random* numbers.  
There is a bunch of functions in `np.random`, for example `normal`,  

```python
np.random.normal(loc=0, scale=1, size=(10, 10))
```

### Exercise: creation routines

- Create an array containing the numbers `[0, 2, 4, 6, 8]` with both `np.arange` and `np.linspace`
- Create an array containing ascending powers of 10 between 0 and one million with both `np.geomspace` and `np.logspace`

In [ ]:
expected = np.array([0, 2, 4, 6, 8])
assert np.array_equal(np.arange(0, 10, 2), expected)
assert np.array_equal(np.linspace(0, 8, 5), expected)

In [ ]:
expected = np.array([1e0, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6])
assert np.array_equal(np.geomspace(1, 1e6, 7), expected)
assert np.array_equal(np.logspace(0, 6, 7), expected)


## Indexing & slicing

Indexing and slicing syntax is the same as with standard Python containers.  
The only difference is that since `numpy` arrays can be multidimensional, you can have multiple slices, separated with a comma.

```python
arr = np.array([[1, 2, 3], [4, 5, 6]])
# array([[1, 2, 3],
#        [4, 5, 6]])

# accessing the rows
arr[0]    # array([1, 2, 3]), equal to arr[0, :]
arr[1]    # array([4, 5, 6]), equal to arr[1, :]

# accessing the columns
arr[:, 0] # array([1, 4])
arr[:, 1] # array([2, 5])
arr[:, 2] # array([3, 6])
```

Note that those slices are **views**, not **copies** of the data, so if you change anything in the slice, the data in the original array is changed as well!  
If you do need a copy, you can use the `.copy()` method on the array view.

```python
arr = np.ones((5, 5))
sub = arr[1:-1, 1:-1]
sub[1, 1] = 2  # changes arr as well!
arr

# array([[1., 1., 1., 1., 1.],
#        [1., 1., 1., 1., 1.],
#        [1., 1., 2., 1., 1.],
#        [1., 1., 1., 1., 1.],
#        [1., 1., 1., 1., 1.]])
``` 


### Exercise: indexing

- Create a 5x5 matrix of ones
- Replace the third column and the third row in the matrix with zeros

In [ ]:
arr = np.ones((5, 5))
arr[:, 2] = np.zeros(5)  # second column
arr[2, :] = np.zeros(5)  # second row

test(
    arr,
    np.array(
        [
            [1.0, 1.0, 0.0, 1.0, 1.0],
            [1.0, 1.0, 0.0, 1.0, 1.0],
            [0.0, 0.0, 0.0, 0.0, 0.0],
            [1.0, 1.0, 0.0, 1.0, 1.0],
            [1.0, 1.0, 0.0, 1.0, 1.0],
        ]
    ),
)



### Exercise: board game layout

- Create a 5x5 array of zeros and then "frame" it with a border of ones

In [ ]:
# boring answer :(
# np.pad(np.zeros((5, 5)), pad_width=1, constant_values=1)

arr = np.ones((7, 7))
arr[1:-1, 1:-1] = np.zeros((5, 5))

test(
    arr,
    np.array(
        [
            [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
            [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
            [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
            [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
            [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
            [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
            [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
        ]
    ),
)


## Reshape, squeeze & flatten 

Many problems in linear algebra and thus machine learning / AI require data to be in a certain *shape*.  
With the `reshape` method you can easily change the layout of your data.  

```python
arr = np.array([1, 2, 3, 4, 5, 6])  # 1d array
# array([1, 2, 3, 4, 5, 6])

arr.reshape(1, 6)                   # row vector
# array([[1, 2, 3, 4, 5, 6]])

arr.reshape(6, 1)                   # column vector
# array([[1],
#        [2],
#        [3],
#        [4],
#        [5],
#        [6]])

arr.reshape(2, 3)
# array([[1, 2, 3],
#        [4, 5, 6]])

arr.reshape(3, 2)
# array([[1, 2],
#        [3, 4],
#        [5, 6]])
```

Do note that a *negative number* in this context means all remaining items.  
So in the example above `arr.reshape(1, 6)` is equal to `arr.reshape(1, -1)`.  
Use the one that you prefer.  

Two special forms of reshape are `flatten` and `squeeze`.  
`flatten` takes a multidimensional array and reduces it to a one-dimensional array

```python
arr = np.array([[1, 2], [3, 4], [5, 6]])
arr.flatten()
# array([1, 2, 3, 4, 5, 6])
```

`squeeze` only removes empty dimensions

```python
arr = np.array([[1, 2, 3, 4, 5, 6]])
arr.squeeze()
# array([1, 2, 3, 4, 5, 6])
```

### Exercise

Create two functions  

`make_row_vector(arr: np.ndarray) -> np.ndarray`  
`make_col_vector(arr: np.ndarray) -> np.ndarray`  

that take an array of any shape and transform it into a row and column vector respectively

In [ ]:
def make_row_vector(arr: NDArray) -> NDArray:
    return arr.reshape(1, -1)

def make_col_vector(arr: NDArray) -> NDArray:
    return arr.reshape(-1, 1)


test(make_row_vector(np.arange(1, 10)).shape, (1, 9))
test(make_col_vector(np.arange(1, 10)).shape, (9, 1))
test(make_row_vector(np.arange(1, 10).reshape(3, 3)).shape, (1, 9))
test(make_col_vector(np.arange(1, 10).reshape(3, 3)).shape, (9, 1))


## Boolean indexing / masking

You can use boolean conditions to select a subset of an array

```python
arr = np.arange(1, 10)
arr[arr > 3]

# array([4, 5, 6, 7, 8, 9])
```

You can also combine multiple conditions, but be sure to always use parentheses around them

```python
arr = np.arange(1, 10)
arr[(arr > 3) & (arr < 6)]
# array([4, 5])
```

### Exercise: boolean indexing

Create an array called `characters` with the same shape as the `numbers` array, filled with the value `" "` (one spacebar).  
For every position in the `numbers` array that is divisible by two, fill the character `"▅"` into the characters array at the same position.  
Use the `special_printer` function to print out the array.  
Does that make you happy?  


In [ ]:
def special_printer(characters: NDArray) -> None:
    for row in characters:
        for val in row:
            print(val, end=" ")
        print()


numbers = np.array(
    [
        [5, 2, 21, 6, 47],
        [33, 1, 11, 23, 45],
        [12, 19, 31, 27, 18],
        [41, 14, 10, 16, 9],
    ]
)

characters = np.full(numbers.shape, fill_value=" ")
characters[numbers % 2 == 0] = "▅"  # U+2585
special_printer(characters)


## Broadcasting vs Linear Algebra

In order to make a lot of common operations easier to write, numpy uses broadcasting (applying an operation to every element in a array).

This works with scalars and arrays

```python
# add scalar to 1d array
v = np.arange(0, 3)
print(v + 2)
# [2 3 4]

# add scalar to 2d array
m = np.arange(0, 9).reshape(3, 3)
print(m + 2)

# [[ 2  3  4]
#  [ 5  6  7]
#  [ 8  9 10]]
```

as well as with multiple arrays

```python
# add 1d array to 2d array
print(np.ones((3, 3)) + np.arange(0, 3))
# [[1. 2. 3.]
#  [1. 2. 3.]
#  [1. 2. 3.]]

# Broadcasting two vectors
print(np.arange(0, 3).reshape(-1, 1) + np.arange(0, 3).reshape(1, -1))
# [[0 1 2]
#  [1 2 3]
#  [2 3 4]]
```


The rules for this are as follows:

- Rule 1: If the two arrays differ in their number of dimensions, the shape of the one with fewer dimensions is padded with ones on its leading (left) side.
- Rule 2: If the shape of the two arrays does not match in any dimension, the array with shape equal to 1 in that dimension is stretched to match the other shape.
- Rule 3: If in any dimension the sizes disagree and neither is equal to 1, an error is raised.

One caveat here is that for **multiplication** you might expect a matrix product, but you actually get a broadcast product. If you *want* to have a matrix product, you have to use the `@` operator

```python
A = np.array([[1, 2], [7, 6]])
B = np.array([[6, 7], [7, 2]])

np.array_equal(A * B, B * A)  # True
np.array_equal(A @ B, B @ A)  # False, as expected
```


### Exercise: transpose

Transposing means flipping over a vector or matrix over it's diagonal and is a very common operation.  
The easiest example is turning a row-vector into a column-vector or the other way around.  


```python
arr = np.array([[1, 2, 3]])
arr
# array([[1, 2, 3]])

arr.T
# array([[1],
#        [2],
#        [3]])
```

Note the two pairs of brackets, which leads to the shape `(1, 3)`.  
Only using `np.array([1, 2, 3])` would lead to the shape `(3, )`, for which the transpose is still `(3, )`!

Using the row vector `v = np.array([[1, 2, 3]])` and the matrix `M = np.array([[1, 2, 3], [4, 5, 6]])`, which are all the products can you build using the `@` operator? Try switching between the normal and transposed version of each of the operands.

In [ ]:
v = np.array([[1, 2, 3]])
M = np.array([[1, 2, 3], [4, 5, 6]])

print(v.T @ v)
print()
print(v @ v.T)
print()
print(M @ v.T)
print()
print(v @ M.T)


## Vectorized functions

Just as you could broadcast `+` in the section above, there are other functions available. 
These are usually referred to as `vectorized` functions, as you don't need to manually use a for loop to apply them to every element.   

| operator | `ufunc`            |
|----------|--------------------|
| `+`      | `np.add`           |
| `-`      | `np.subtract`      |
| `*`      | `np.multiply`      |
| `/`      | `np.divide`        |
| `/`/     | `np.floor_divide`  |
| `*`*     | `np.power`         |
| `%`      | `np.mod`           |
| `==`     | `np.equal`         |
| `!=`     | `np.not_equal`     |
| `<`      | `np.less`          |
| `<=`     | `np.less_equal`    |
| `>`      | `np.greater`       |
| `>=`     | `np.greater_equal` |

A (non-exhaustive) list of useful numpy `ufuncs`.

- `np.sum`
- `np.prod`
- `np.min`
- `np.max`
- `np.abs`
- `np.mean`
- `np.median`
- `np.var`
- `np.std`
- `np.exp`
- `np.log`
- `np.sin`
- `np.cos`
- `np.tan`
- `np.arcsin`
- `np.arccos`
- `np.arctan`

You can explore other functions using `.<TAB>` autocompletion or via the official [documentation](https://numpy.org/doc/stable/reference/ufuncs.html)

### Exercise: root mean square error

The RMSE is defined as the square root of the mean of the squared error between a prediction $\hat{y_i}$ and the correct value $y_i$.

$$ \sqrt{\frac{\Sigma_{i=0}^N (\hat{y_i} - y_i)^2}{N} }$$

Implement the root mean square error as a function.

In [ ]:
def rmse(y_pred: NDArray[np.float64], y_true: NDArray[np.float64]) -> float:
    return np.sqrt(np.mean(np.square(y_pred - y_true)))


test(rmse(np.array([1, 2, 3]), np.array([2, 3, 4])), 1.0)


## Combining & splitting arrays

If you are combining data from multiple sources, or are trying to extract data it's very helpful to combine and split arrays.  

Let's start with two column-vectors

```python
x = np.array([1, 2, 3]).reshape(-1, 1)
y = np.array([4, 5, 6]).reshape(-1, 1)
```

We can stack these two horizontally

```python
z_h = np.hstack([x, y])
# array([[1, 4],
#        [2, 5],
#        [3, 6]])
```

or vertically

```python
z_v = np.vstack([x, y])
# array([[1],
#        [2],
#        [3],
#        [4],
#        [5],
#        [6]])
```

Similarly, we can split the two arrays again. Do not that you need to specify in how many pieces you split them

```python
np.hsplit(z_h, 2)
np.vsplit(z_v, 2)
```

There is also an `np.split` function, but I would suggest to only use that for one-dimensional arrays, as the other two function names make your intentions much clearer.


### Exercise

The following array is badly sorted. Using splitting and stacking, create a sorted version of it

In [ ]:
arr = np.array(
    [
        [4, 5, 6, 7],
        [12, 13, 14, 15],
        [0, 1, 2, 3],
        [8, 9, 10, 11],
    ]
)


a, b, c, d = np.vsplit(arr, 4)
sorted_arr = np.vstack([c, a, d, b])

test(
    sorted_arr,
    np.array(
        [
            [0, 1, 2, 3],
            [4, 5, 6, 7],
            [8, 9, 10, 11],
            [12, 13, 14, 15],
        ]
    ),
)


## Further learning

Documentation

- [numpy documentation](https://numpy.org/doc/stable/user/index.html#user)
- [pandas documentation](https://pandas.pydata.org/docs/user_guide/index.html)
- [matplotlib documentation](https://matplotlib.org/stable/index.html)

Further packages

- [seaborn](https://seaborn.pydata.org/): package built on top of matplotlib for statistical plots
- [SciPy](https://docs.scipy.org/doc/scipy/tutorial/index.html#user-guide): advanced scientific computing library
- [statsmodels](https://www.statsmodels.org/stable/index.html): statistical models
- [scikit-learn](https://scikit-learn.org/stable/): machine learning library
- [PyTorch](https://pytorch.org/docs/stable/index.html): deep learning library
- [tensorflow](https://www.tensorflow.org/tutorials): deep learning library
- [Keras](https://keras.io/): deep learning library 
- [aesara](https://github.com/aesara-devs/aesara) (used to be Theano): symbolic maths on multi-dimensional arrays
- [JAX](https://github.com/google/jax): Composable transformations of Python & numpy on GPUs 

Books

- [Jake VanderPlas - Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/)
- [Wes McKinnery - Python for Data Analysis](https://wesmckinney.com/book/)